In [1]:
!which python

/sw/centos/anaconda3/2019.10/bin/python


In [2]:
import csv
import numpy as np
from collections import Counter
from nltk.corpus import brown
from mittens import GloVe, Mittens
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import re
import json
import os
import pandas as pd
import spacy
import nltk
from nltk.tokenize import sent_tokenize

In [3]:
def glove2dict(glove_filename):
    with open(glove_filename, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed

def get_rareoov(xdict, val):
    return [k for (k,v) in Counter(xdict).items() if v<=val]

# Load Data

In [4]:
# Load the orignial dataset to train glove. Larger dataset can give better results for word embedding
dir_path = '../Dataset/ratebeer/original_dataset'
# Load train dataset
train_review = []
cnt = 0
file_path = os.path.join(dir_path, 'train_no_duplicate.json')
with open(file_path) as f:
    for line in f:
        line_data = json.loads(line)
        user_id = line_data['user']
        item_id = line_data['item']
        rating = line_data['rating']
        review = line_data['review']
        train_review.append([item_id, user_id, rating, review])
        cnt += 1
        if cnt % 100000 == 0:
            print('{} lines loaded.'.format(cnt))
print('Finish loading train dataset, totally {} lines.'.format(len(train_review)))

100000 lines loaded.
200000 lines loaded.
300000 lines loaded.
400000 lines loaded.
500000 lines loaded.
600000 lines loaded.
700000 lines loaded.
800000 lines loaded.
900000 lines loaded.
Finish loading train dataset, totally 918465 lines.


In [4]:
dir_path = '../Dataset/ratebeer/medium_30'
# Load train dataset
train_review = []
cnt = 0
file_path = os.path.join(dir_path, 'train_review_filtered.json')
with open(file_path) as f:
    for line in f:
        line_data = json.loads(line)
        user_id = line_data['user']
        item_id = line_data['item']
        rating = line_data['rating']
        review = line_data['review']
        train_review.append([item_id, user_id, rating, review])
        cnt += 1
        if cnt % 100000 == 0:
            print('{} lines loaded.'.format(cnt))
print('Finish loading train dataset, totally {} lines.'.format(len(train_review)))
# Load test dataset
# test_review = []
# cnt = 0
# file_path = os.path.join(dir_path, 'test_review_filtered.json')
# with open(file_path) as f:
#     for line in f:
#         line_data = json.loads(line)
#         user_id = line_data['user']
#         item_id = line_data['item']
#         rating = line_data['rating']
#         review = line_data['review']
#         test_review.append([item_id, user_id, rating, review])
#         cnt += 1
#         if cnt % 10000 == 0:
#             print('{} lines loaded.'.format(cnt))
# print('Finish loading test dataset, totally {} lines.'.format(len(test_review)))

100000 lines loaded.
Finish loading train dataset, totally 117818 lines.


In [5]:
df_train_data = pd.DataFrame(train_review, columns=['item', 'user', 'rating', 'review'])
# df_test_data = pd.DataFrame(test_review, columns=['item', 'user', 'rating', 'review'])

In [6]:
train_review_doc = list(df_train_data['review'])

In [7]:
len(train_review_doc)

918465

In [8]:
with open('./embeddings/whole/train-review.txt', 'w') as f:
    cnt = 0
    for review in train_review_doc:
        # write this into file
        f.write(review.strip())
        f.write('\n')
        cnt += 1
    print('Write {} lines of review into file'.format(cnt))

Write 918465 lines of review into file


In [10]:
from collections import Counter
import simplejson
from nltk.tokenize import word_tokenize
import string
import re
import nltk
nltk.download('punkt')

def tokenize(infname="embeddings/whole/train-review.txt"):
    outfname = open(infname + ".tokenized", "w")
    
    # ----------------------------------------
    # TODO: add your code here
    sents = open(infname).read().split("\n")
    for sent in sents:
        # convert characters into lower cases
        text = sent.lower()
        # tokenize the raw text using nltk.tokenize
        text = word_tokenize(text)
        # remove all punctuation
        text = list(filter(lambda token: token not in string.punctuation, text))
        # # replace all numbers with a special token <num>
        # text = [re.sub('[0-9]+','<num>', token) for token in text]
        # write this processed text into the output file
        simplejson.dump(text, outfname)
        outfname.write("\n")
    # ----------------------------------------
    outfname.close()


def token_filter(infname="embeddings/whole/train-review.txt.tokenized", thresh=5):
    outfname = open(infname.replace(".tokenized", ".filtered"), 'w')
    vocab = []  
    # ----------------------------------------
    # TODO: remove "pass" and add your code here
    vocab_cnt = Counter()
    tokenized_sents = []
    with open(infname) as f_in:
        for line in f_in:
            tokenized_text = simplejson.loads(line)
            vocab_cnt.update(tokenized_text)
            tokenized_sents.append(tokenized_text)
    for tokenized_sent in tokenized_sents:
        # filter word that has less than 5 TF over the whole dataset
        filtered_text = []
        for token in tokenized_sent:
            if vocab_cnt[token] < thresh:
                pass
            else:
                filtered_text.append(token)
        # write this filtered data into file
        # ignore the last line of the initial imdb-small dataset since it's an empty line
        if len(tokenized_sent) > 0:
            new_sentence = " ".join(filtered_text).strip()+'\n'
            outfname.write(new_sentence)
    # bulld the vocabulary
    ## filter out token with low TF
    vocab_cnt_filter = {token: cnt for token, cnt in vocab_cnt.items() if cnt >= thresh}
    vocab_cnt_filter = Counter(vocab_cnt_filter)
    ## convert this vocabulary counter to a list
    for token,cnt in vocab_cnt_filter.most_common():
        vocab.append(token)
    # ----------------------------------------
    outfname.close()
    return vocab

[nltk_data] Downloading package punkt to /u/pw7nc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
tokenize()
vocab = token_filter()
print("The vocab size = {}".format(len(vocab)))

The vocab size = 43670


## Load Feature Words

In [ ]:
feature_2_id_file = '../Dataset/ratebeer/medium_500/train/feature/feature2id.json'
with open(feature_2_id_file, 'r') as f:
    feature_vocab = json.load(f)

In [14]:
feature_2_id_file_large = '../Dataset/ratebeer/large/train/feature/feature2id.json'
with open(feature_2_id_file_large, 'r') as f:
    feature_vocab_large = json.load(f)

In [16]:
# Make Sure that all feature words are in the vocab
for fea_word in list(feature_vocab_large.keys()):
    if fea_word in vocab:
        pass
    else:
        print(fea_word)

gotta


In [17]:
len(feature_vocab_large)

5000

In [18]:
total_vocab = set(vocab) | set(list(feature_vocab_large.keys()))

In [40]:
len(total_vocab)

43671

In [20]:
total_vocab_list = list(total_vocab)

In [21]:
# sw = list(_stop_words.ENGLISH_STOP_WORDS)
# brown_data = brown.words()[:200000]
# brown_nonstop = [token.lower() for token in brown_data if (token.lower() not in sw)]



# corp_vocab = list(set(oov))
# brown_doc = [' '.join(brown_nonstop)]

cv = CountVectorizer(ngram_range=(1,1), vocabulary=total_vocab_list)
X = cv.fit_transform(train_review_doc)
Xc = (X.T * X)
Xc.setdiag(0)
coocc_ar = Xc.toarray()

/sw/centos/anaconda3/2019.10/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [22]:
coocc_ar.shape

(43671, 43671)

In [23]:
X.shape

(918465, 43671)

In [24]:
len(cv.vocabulary_)

43671

In [25]:
train_vocab = cv.get_feature_names()

In [26]:
train_vocab[:20]

['punctuated',
 'tails',
 '12oz',
 'grapefriut',
 'alaskan',
 'transform',
 'rivertown',
 'farts',
 'proably',
 'paisse',
 'potpouri',
 'shits',
 'nitrous',
 'overides',
 'charachter',
 'traverse',
 'overweight',
 'devonshire',
 'bubles',
 'rockier']

In [27]:
cv.vocabulary_["aroma"]

13844

In [28]:
len(train_vocab)

43671

In [29]:
train_vocab == total_vocab_list

True

In [31]:
from mittens import GloVe
import numpy as np

# mittens_model = Mittens(n=256, max_iter=1000)

# new_embeddings = mittens_model.fit(
#     coocc_ar,
#     vocab=train_vocab)
glove_model = GloVe(n=256, max_iter=1000)
embeddings = glove_model.fit(coocc_ar)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Iteration 1000: loss: 446263.09375

In [35]:
embeddings.shape

(43671, 256)

In [36]:
len(cv.vocabulary_)

43671

In [37]:
len(cv.get_feature_names())

43671

In [38]:
vocabulary_words = cv.get_feature_names()

In [39]:
vocabulary_words[:20]

['punctuated',
 'tails',
 '12oz',
 'grapefriut',
 'alaskan',
 'transform',
 'rivertown',
 'farts',
 'proably',
 'paisse',
 'potpouri',
 'shits',
 'nitrous',
 'overides',
 'charachter',
 'traverse',
 'overweight',
 'devonshire',
 'bubles',
 'rockier']

In [42]:
# construct an embedding dict which has a format of {'word':'aroma', 'embedding': array([...])}
word_embedding_dict = dict()
for i in range(len(embeddings)):
    current_word = vocabulary_words[i]
    current_embedding = embeddings[i]
    word_embedding_dict[current_word] = current_embedding
print("Totally {} words begin added in the word embedding dict".format(len(word_embedding_dict)))

Totally 43671 words begin added in the word embedding dict


In [4]:
# Save the Embedding dict into pickle file
import pickle
with open('../Dataset/ratebeer/embedding/glove.pickle', 'wb') as handle:
    pickle.dump(word_embedding_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

NameError: name 'word_embedding_dict' is not defined

In [46]:
# extract the embedding vector for each feature word
feature_embedding = dict()
emb_cnt = 0
for feature_word in list(feature_vocab.keys()):
    feature_word_id = cv.vocabulary_[feature_word]
    feature_vocab_id = feature_vocab[feature_word]
    feature_word_emb = embeddings[feature_word_id]
    feature_embedding[feature_vocab_id] = feature_word_emb
    emb_cnt += 1
print("Totally {} feature words".format(emb_cnt))

Totally 1000 feature words


In [48]:
# extract the embedding vector for each feature word
feature_embed_dict = dict()
for emb_chunk in list(feature_embedding.items()):
    feature_id = emb_chunk[0]   # this is a str
    feature_emb_np = emb_chunk[1].tolist()
    feature_embed_dict[feature_id] = feature_emb_np

In [ ]:
# feature_embed_dict['0']

In [49]:
with open('../Dataset/ratebeer/medium_30/train/feature/featureid2embedding_wholeglove.json', 'w') as f:
    json.dump(feature_embed_dict, f)

# Load Previously Trained Feature Embedding

In [29]:
with open('../Dataset/ratebeer/medium_30/train/feature/featureid2embedding.json', 'r') as f:
    feature_embed_dict_large = json.load(f)

In [30]:
len(feature_embed_dict_large)

5000

In [32]:
# make sure that all the feature words in the medium feature words vocab is in the large feature word vocab
assert len(feature_vocab.keys()) == 1000
for feature_word_medium in feature_vocab.keys():
    if feature_word_medium in feature_vocab_large:
        pass
    else:
        print(feature_word_medium)

In [34]:
# get the embedding vector for each feature word in the medium dataset
feature_embed_dict_medium = dict()
cnt = 0
for feature_word_medium in feature_vocab.keys():
    # get the id of the feature word
    fea_id = feature_vocab[feature_word_medium]
    assert fea_id == str(cnt)
    cnt += 1
    # get the id of the feature word in the large dataset
    fea_id_large = feature_vocab_large[feature_word_medium]
    assert isinstance(fea_id_large, str)
    # get the emebdding
    fea_embedding = feature_embed_dict_large[fea_id_large]
    # save the feature and its embedding into the dict
    feature_embed_dict_medium[fea_id] = fea_embedding

In [35]:
len(feature_embed_dict_medium)

1000

In [38]:
len(feature_embed_dict_medium['0'])

256

In [5]:
import json
with open("../Dataset/ratebeer/medium_500/train/feature/featureid2embedding.json", 'r') as f:
    feature_to_embedding = json.load(f)

In [6]:
len(feature_to_embedding)

1000

In [9]:
len(feature_to_embedding['999'])

256